In [20]:
import pandas as pd
import numpy as np

In [21]:
X = pd.read_csv("./data.csv", header=None).iloc[:, :-1].values

In [22]:
def gaussian_kernel(x):
    d = x.shape[0]
    return (1 / (2 * np.pi) ** (d / 2)) * np.exp(-0.5 * np.dot(x, x))

def multivariate_kde(X, H):
    n, d = X.shape
    Hinv = np.linalg.inv(H)
    Hdet = np.linalg.det(H)
    
    def kde(x):
        x = np.asarray(x)
        if x.ndim == 1:
            x = x.reshape(1, -1)
        
        N = x.shape[0]
        result = np.zeros(N)
        
        for i in range(N):
            diff = x[i] - X
            tdiff = np.dot(diff, Hinv)
            energy = np.sum(diff * tdiff, axis=1)
            result[i] = np.sum(np.exp(-0.5 * energy))
        
        result /= (n * Hdet ** 0.5)
        return result[0]
    
    return kde

In [23]:
def scotts_rule(X):
    n, d = X.shape
    sigma = np.std(X, axis=0)
    h = n ** (-1 / (d + 4)) * sigma
    return np.diag(h)

In [24]:
def numerical_gradient(f, x, h=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_plus = x.copy()
        x_plus[i] += h
        x_minus = x.copy()
        x_minus[i] -= h
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * h)
    return grad

def find_local_maxima(kde, X, num_restarts=10, max_iter=100, tol=1e-1, step_size=0.1):
    n, d = X.shape
    local_maxima = []
    
    for _ in range(num_restarts):
        x = X[np.random.choice(n)]
        for _ in range(max_iter):
            grad = numerical_gradient(kde, x)
            x_new = x + step_size * grad
            if np.linalg.norm(x_new - x) < tol:
                break
            x = x_new
        local_maxima.append(x)
    
    return np.unique(np.array(local_maxima).round(decimals=-int(np.log10(tol))), axis=0)

In [42]:
kde = multivariate_kde(X, scotts_rule(X))
local_max = find_local_maxima(kde, X)
local_max

array([[4.9, 2.5, 4.5, 1.7],
       [5. , 2. , 3.5, 1. ],
       [5.4, 3.7, 1.5, 0.2],
       [5.4, 3.8, 1.3, 0.4],
       [6. , 3. , 4.8, 1.7],
       [6.3, 2.9, 5.6, 1.8],
       [6.3, 3.1, 5.4, 2.1],
       [6.4, 2.8, 5.6, 2.1],
       [6.7, 3.3, 5.7, 2.5],
       [7.3, 2.9, 6.3, 1.8]])

In [26]:
local_max[np.random.choice(len(local_max), size=4, replace=False)]

array([[6.6, 3. , 4.7, 1.5],
       [5.5, 2.4, 3.7, 1. ],
       [5.5, 2.4, 3.8, 1.1],
       [5.2, 3.4, 1.4, 0.2]])

# Testing

In [44]:
from kde_kernel import initiate_kde
import pandas as pd
import numpy as np

In [73]:
X = pd.read_csv("./data.csv", header=None).iloc[:, :-1].values
np.sort(initiate_kde(X,3), axis=0)

array([[5.1, 2.4, 1.5, 0.2],
       [5.5, 2.5, 3.8, 1.1],
       [6.7, 3.4, 5.8, 1.8]])